In [ ]:
import numpy as np
import cupy as cp  # Only if you have a GPU
from scipy.ndimage import distance_transform_edt
from skimage.morphology import skeletonize_3d
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import skan.csr

def calculate_local_thickness(binary_volume, skeleton_volume, max_distance=10):
    """
    Calculate a masked distance map near the skeleton.
    """
    # Calculate distance only around the skeleton to reduce computation
    masked_distance_map = distance_transform_edt(binary_volume)
    masked_distance_map[masked_distance_map > max_distance] = 0  # Mask distances above a threshold
    return masked_distance_map * skeleton_volume

def compute_statistics(skeleton):
    """
    Computes mean, max, and median widths for each path in parallel.
    """
    def path_stats(path_id):
        widths = skeleton.path_with_data(path_id)[1]
        return np.mean(widths), np.max(widths), np.median(widths)
    
    # Use parallel processing to calculate statistics
    results = Parallel(n_jobs=-1)(delayed(path_stats)(i) for i in range(skeleton.n_paths))
    means, maxs, medians = zip(*results)
    return means, maxs, medians

# Assuming `binary_volume` is the binary image of the vascular structure
# Step 1: Skeletonize the volume
skeleton_volume = skeletonize_3d(binary_volume)

# Step 2: Compute a masked local thickness map for the skeleton
masked_thickness_map = calculate_local_thickness(binary_volume, skeleton_volume)

# Step 3: Initialize Skeleton with the masked thickness data
skeleton = Skeleton(skeleton_volume, skeleton_image=masked_thickness_map)

# Step 4: Summarize and add custom columns for width statistics
summary = summarize(skeleton)

# Calculate mean, max, and median width for each path in parallel
mean_width, max_width, median_width = compute_statistics(skeleton)

# Add to summary DataFrame
summary['mean_width'] = mean_width
summary['max_width'] = max_width
summary['median_width'] = median_width

print(summary)
